In [2]:
import pandas as pd

# Load the CSV file
file_path = '/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/Prediction-Youth-and-Tobacco/nyts2023.csv'
df = pd.read_csv(file_path)

# Display the first few rows of the dataset
df.head()




Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/myenv/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3508, in run_code
  File "/var/folders/4z/n7sr27qx5h18n91g81tmnsdh0000gn/T/ipykernel_40744/716422294.py", line 5, in <module>
    df = pd.read_csv(file_path)
  File "/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/myenv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 912, in read_csv
  File "/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/myenv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 577, in _read
  File "/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/myenv/lib/python3.8/site-packages/pandas/io/parsers/readers.py", line 1407, in __init__
  File "/Users/lucasliu/Desktop/483data/2023-nyts-dataset-codebook-microsoft-excel/myenv/lib/python3.8/site-packages/pandas/io/parsers/readers